Generating Training and Test data

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random

# make x train data and y result , I for input data num , T for x time
I_train = 30 
T_train = 8

random_x_train = np.random.binomial(n=1,p=0.5,size=(I_train,T_train))
random_x_train

random_y_train = random_x_train.sum(axis=1).reshape(-1,1)

#make random x test data and y test result
I_test = 10
T_test = 8

random_x_test = np.random.binomial(n=1,p=0.5,size=(I_test,T_test))
random_y_test = random_x_test.sum(axis=1).reshape(-1,1)

random_x_test , random_y_test
random_x_train , random_y_train

(array([[0, 0, 0, 1, 0, 1, 1, 1],
        [1, 1, 1, 1, 1, 0, 1, 1],
        [1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 1, 1, 1],
        [0, 0, 1, 0, 1, 1, 1, 0],
        [1, 1, 1, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 1, 0, 1, 0],
        [1, 0, 1, 1, 0, 0, 0, 0],
        [1, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 1, 1, 0, 1, 1, 0],
        [1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 1],
        [1, 0, 1, 1, 1, 0, 1, 0],
        [0, 0, 1, 1, 1, 1, 1, 0],
        [1, 0, 1, 1, 0, 1, 0, 0],
        [1, 0, 1, 0, 0, 0, 1, 1],
        [1, 1, 0, 1, 0, 0, 1, 0],
        [1, 1, 0, 1, 1, 1, 0, 1],
        [1, 0, 1, 0, 0, 1, 0, 1],
        [0, 1, 1, 1, 0, 1, 1, 0],
        [0, 0, 1, 1, 0, 0, 1, 1],
        [1, 0, 0, 1, 0, 0, 1, 1],
        [1, 1, 1, 0, 1, 1, 1, 0],
        [1, 0, 0, 1, 1, 1, 0, 0],
        [0, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 0, 1, 0, 1, 1],
        [1, 1, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 1, 0, 0, 0, 1],
        [0, 1, 1, 1, 0, 0, 1, 0],
        [0, 1,

Define Functions

In [0]:
def initialize_f_0(f_0,input_arr):
  f_0_column = np.full((len(input_arr),1),f_0)
  return f_0_column

def makeinput(input_arr,index,f_0):
  f_0_arr=initialize_f_0(f_0,input_arr)
  x_mat_vector = input_arr[:,index]
  x_mat = x_mat_vector.reshape(-1,1)
  concat_mat_2 = np.hstack([x_mat,f_0_arr])
  return concat_mat_2

def updateinput(input_arr,index,f_t_arr):
  x_mat_vector = input_arr[:,index]
  x_mat = x_mat_vector.reshape(-1,1)
  concat_mat_2 = np.hstack([x_mat,f_t_arr])
  return concat_mat_2

def get_f_t(input_arr , parameter_v):
  f_t = np.matmul(input_arr, parameter_v)
  return f_t

def get_f_t_arr(f_0,input_arr,parameter_v,initial_arr):
  f_0_column = np.full((len(input_arr),1),f_0)
  for i in range(1,len(input_arr[0])):
    f_t_arr = get_f_t(initial_arr , parameter_v)
    initial_arr = updateinput(input_arr,i,f_t_arr)
    f_0_column = np.hstack([f_0_column,f_t_arr])
  f_t_finarr = get_f_t(initial_arr , parameter_v)
  f_0_column = np.hstack([f_0_column,f_t_finarr])
  return f_0_column

def doforwardpropagation(f_0,input_arr,parameter_v,initial_arr):
  f_0_column = np.full((len(input_arr),1),f_0)
  for i in range(1,len(input_arr[0])):
    f_t_arr = get_f_t(initial_arr , parameter_v)
    initial_arr = updateinput(input_arr,i,f_t_arr)
    f_0_column = np.hstack([f_0_column,f_t_arr])
  f_t_finarr = get_f_t(initial_arr , parameter_v)
  y_hat = 1*f_t_finarr
  return y_hat  

def do_err_derivative(input_arr, f_t_arr, parameter_v, y_hat, y):
  sum_t = np.zeros(2) 
  for n_1_k in range(0,2):
    for t in range(0,len(input_arr[0])):
      sum_i = 0
      for i in range(0,len(input_arr)):
        if n_1_k == 0:
          sum_i +=  (y_hat[i,0]-y[i,0])*input_arr[i,t]
        else:
          sum_i +=  (y_hat[i,0]-y[i,0])*f_t_arr[i,t]
      sum_t[n_1_k] += sum_i*(parameter_v[1,0]**(len(input_arr[0])-t))      
  err_der_arr = sum_t
  return err_der_arr    

def do_backpropagation(parameter_v, alpha, err_derivative):
  for n_1_k in range(0,len(parameter_v)):
    parameter_v[n_1_k,0] =   parameter_v[n_1_k,0] - alpha*err_derivative[n_1_k]
  return parameter_v  
 
def update_delta(prev_err_der, now_err_der, delta_x, delta_f, n_p, n_n):
  if np.sign(prev_err_der[0]) == np.sign(now_err_der[0]):
    delta_x = delta_x*n_p
  else:
    delta_x = delta_x*n_n
      
  if np.sign(prev_err_der[1]) == np.sign(now_err_der[1]):
    delta_f = delta_f*n_p
  else:
    delta_f = delta_f*n_n  
      
  return delta_x, delta_f 
  
def do_resilient_x(prev_der, now_der, n_p, n_n, delta_x, parameter_v):
  if np.sign(prev_der[0]) == np.sign(now_der[0]):
    delta_x = n_p*delta_x
    parameter_v[0,0] = parameter_v[0,0] -1*np.sign(now_der[0])*delta_x
  else:
    delta_x = n_n*delta_x
    parameter_v[0,0] = parameter_v[0,0] -1*np.sign(now_der[0])*delta_x
  return parameter_v[0,0], delta_x

def do_resilient_f(prev_der, now_der, n_p, n_n, delta_f, parameter_v):
  if np.sign(prev_der[0]) == np.sign(now_der[0]):
    delta_f = n_p*delta_f
    parameter_v[1,0] = parameter_v[1,0] -1*np.sign(now_der[1])*delta_f
  else:
    delta_f = n_n*delta_f
    parameter_v[1,0] = parameter_v[1,0] -1*np.sign(now_der[1])*delta_f
  return parameter_v[1,0], delta_f


def do_gradient_clip_x(err_der , threshold_n, parameter_v ,alpha):
  if np.abs(err_der[0]) > threshold_n:
    err_der[0] = err_der[0]*threshold_n/np.abs(err_der[0])
  else:
    err_der[0] = err_der[0]
  parameter_v[0,0] = parameter_v[0,0] - alpha*err_der[0]
  return parameter_v[0,0]

def do_gradient_clip_f(err_der , threshold_n, parameter_v ,alpha):
  if np.abs(err_der[1]) > threshold_n:
    err_der[1] = err_der[1]*threshold_n/np.abs(err_der[1])
  else:
    err_der[1] = err_der[1]
  parameter_v[1,0] = parameter_v[1,0] - alpha*err_der[1]
  return parameter_v[1,0]

def geterr(y_hat, y):
  value1 = 0
  for i in range(0,len(y)):
    value1 += (y_hat[i,0]-y[i,0])**2
  err = 0.5*value1    
  return err
    

Initializing  parameters

In [27]:
#backpropagation
learningrate_v_x = 0.01
learningrate_v_f = 0.01
learningrate = 0.00001

#resilient propagation
delta_x = 0.001
delta_f = 0.001
n_p = 1.2 
n_n = 0.5

#for main loop
E_present = 10
E_previous = 15
e = 0.000001

np.random.seed(4)

#for v
#parameter_v = np.random.randint(10,size=(2,1))
parameter_v = 2*np.random.rand(2,1)
parameter_v_x = parameter_v[0]
parameter_v_f = parameter_v[1]
parameter_v

array([[1.93405968],
       [1.0944645 ]])

Backpropagation

In [28]:
f_0 = 1
new_parameter_v = parameter_v
init_input = makeinput(random_x_train,0,f_0)

#main loop
while(np.abs(E_present-E_previous)>e):
#for epoch in range(0,5):  
  f_t_arr = get_f_t_arr(f_0, random_x_train, new_parameter_v, init_input)
  y_hat = doforwardpropagation(f_0,random_x_train, new_parameter_v, init_input)
  
  E_previous = E_present
  E_present = geterr(y_hat,random_y_train)
  
  err_der_arr = do_err_derivative(random_x_train, f_t_arr, new_parameter_v, y_hat,random_y_train)
  new_parameter_v = do_backpropagation(new_parameter_v, learningrate, err_der_arr)  
  print(E_present)
  print(new_parameter_v)
  
print(parameter_v_x, parameter_v_f)  
  

1465.4710888305397
[[1.91342834]
 [0.89212582]]
47.28391022477445
[[1.91203111]
 [0.88187852]]
36.47274709242238
[[1.91087741]
 [0.87370407]]
29.41161407974191
[[1.90989747]
 [0.86698437]]
24.543318239002758
[[1.90904794]
 [0.86134103]]
21.053602496759588
[[1.90829988]
 [0.85652494]]
18.47738347029185
[[1.90763298]
 [0.85236317]]
16.53121576866519
[[1.90703238]
 [0.84873078]]
15.033524534706558
[[1.90648689]
 [0.84553447]]
13.863297587749969
[[1.90598784]
 [0.84270279]]
12.937245667970014
[[1.9055284 ]
 [0.84017983]]
12.19649072899673
[[1.90510308]
 [0.837921  ]]
11.5984634179854
[[1.90470739]
 [0.83589018]]
11.11178843155063
[[1.90433763]
 [0.83405777]]
10.712954280390024
[[1.90399069]
 [0.83239916]]
10.384086158978475
[[1.90366397]
 [0.83089371]]
10.111421418527682
[[1.90335523]
 [0.82952394]]
9.884244389290947
[[1.90306256]
 [0.82827495]]
9.694128513723905
[[1.90278432]
 [0.8271339 ]]
9.534388326490191
[[1.90251905]
 [0.82608968]]
9.39967737680456
[[1.90226551]
 [0.82513264]]
9.2856

Resilient propagation

In [32]:
f_0 = 1
np.random.seed(0)
#res_parameter_v = np.random.randint(10,size=(2,1))
res_parameter_v = 2*np.random.rand(2,1)
print(res_parameter_v)
res_input = makeinput(random_x_train,0,f_0)
E_present_res = 10
E_previous_res = 15
now_err_der_res = np.random.rand(2)
prev_err_der_res = np.random.rand(2)

#main loop
while(np.abs(E_present_res-E_previous_res)>0.0000001):
#for epoch in range(0,5):  
  res_f_t_arr = get_f_t_arr(f_0, random_x_train, res_parameter_v, res_input)
  res_y_hat = doforwardpropagation(f_0,random_x_train, res_parameter_v, res_input)
  
  
  E_previous_res = E_present_res
  E_present_res = geterr(res_y_hat,random_y_train)
  
  prev_err_der_res = now_err_der_res
  now_err_der_res = do_err_derivative(random_x_train, res_f_t_arr, res_parameter_v, res_y_hat,random_y_train)
  res_parameter_v[0,0], delta_x = do_resilient_x(prev_err_der_res, now_err_der_res, n_p, n_n,delta_x, res_parameter_v)
  res_parameter_v[1,0], delta_f = do_resilient_f(prev_err_der_res, now_err_der_res, n_p, n_n,delta_f, res_parameter_v)
  
 # print(E_present_res)
 # print(res_parameter_v)
  
print(res_y_hat)
print(random_y_train)
  

[[1.09762701]
 [1.43037873]]
[[4.31107564]
 [6.59598726]
 [3.90760549]
 [6.62372615]
 [4.19909862]
 [3.08578939]
 [3.32268269]
 [3.13698772]
 [3.98566475]
 [4.17223767]
 [3.90760549]
 [3.43465971]
 [4.89072621]
 [5.02091471]
 [4.01340364]
 [4.15487635]
 [4.01686164]
 [5.77153642]
 [4.12623082]
 [4.94285545]
 [4.23046502]
 [4.18088714]
 [5.71594382]
 [4.06627537]
 [4.88647102]
 [5.77417116]
 [3.22462736]
 [3.32540355]
 [4.06643952]
 [3.29749516]]
[[4]
 [7]
 [4]
 [7]
 [4]
 [3]
 [3]
 [3]
 [4]
 [4]
 [4]
 [3]
 [5]
 [5]
 [4]
 [4]
 [4]
 [6]
 [4]
 [5]
 [4]
 [4]
 [6]
 [4]
 [5]
 [6]
 [3]
 [3]
 [4]
 [3]]


Gradient clipping

In [9]:
f_0 = 1
np.random.seed(0)
#gc_parameter_v = np.random.randint(10,size=(2,1))
gc_parameter_v = 2*np.random.rand(2,1)
print(gc_parameter_v)
gc_input = makeinput(random_x_train,0,f_0)
E_present_gc = 10
E_previous_gc = 15
threshold_n = 1
learningrate_gc = 0.00001

#main loop
while(np.abs(E_present_gc-E_previous_gc)>e):
#for epoch in range(0,5):  
  gc_f_t_arr = get_f_t_arr(f_0, random_x_train, gc_parameter_v, gc_input)
  gc_y_hat = doforwardpropagation(f_0,random_x_train, gc_parameter_v, gc_input)
  
  E_previous_gc = E_present_gc
  E_present_gc = geterr(gc_y_hat,random_y_train)
  
  err_der_gc = do_err_derivative(random_x_train, gc_f_t_arr, gc_parameter_v, gc_y_hat,random_y_train)
  gc_parameter_v[0,0] = do_gradient_clip_x(err_der_gc, threshold_n, gc_parameter_v, learningrate_gc)
  gc_parameter_v[1,0] = do_gradient_clip_f(err_der_gc, threshold_n, gc_parameter_v, learningrate_gc)
  
  print(E_present_gc)
  print(gc_parameter_v)
  
  

Output hidden; open in https://colab.research.google.com to view.

backprop test

In [18]:
f_0 = 1
new_parameter_v = parameter_v
init_input = makeinput(random_x_test,0,f_0)

#main loop
#while(np.abs(E_present-E_previous)>e):
#for epoch in range(0,5):  
f_t_arr = get_f_t_arr(f_0, random_x_test, new_parameter_v, init_input)
y_hat = doforwardpropagation(f_0,random_x_test, new_parameter_v, init_input)
  
E_previous = E_present
E_present = geterr(y_hat,random_y_test)
  
 # err_der_arr = do_err_derivative(random_x_test, f_t_arr, new_parameter_v, y_hat,random_y_test)
  #new_parameter_v = do_backpropagation(new_parameter_v, learningrate, err_der_arr)  
print(E_present)
print(new_parameter_v)
  
print(parameter_v_x, parameter_v_f)  
  

nan
[[nan]
 [nan]]
[nan] [nan]


resilient test

In [23]:
f_0 = 1

#res_parameter_v = np.random.randint(10,size=(2,1))
#res_parameter_v_test = res_parameter  #2*np.random.rand(2,1)
print(res_parameter_v)
res_input = makeinput(random_x_test,0,f_0)
E_present_res = 10
E_previous_res = 15
now_err_der_res = np.random.rand(2)
prev_err_der_res = np.random.rand(2)

#main loop
#while(np.abs(E_present_res-E_previous_res)>e):
#for epoch in range(0,5):  
res_f_t_arr = get_f_t_arr(f_0, random_x_test, res_parameter_v, res_input)
res_y_hat = doforwardpropagation(f_0,random_x_test, res_parameter_v, res_input)
  
E_previous_res = E_present_res
E_present_res = geterr(res_y_hat,random_y_test)
  
 # prev_err_der_res = now_err_der_res
 # now_err_der_res = do_err_derivative(random_x_test, res_f_t_arr, res_parameter_v, res_y_hat,random_y_test)
 # res_parameter_v[0,0], delta_x = do_resilient_x(prev_err_der_res, now_err_der_res, n_p, n_n,delta_x, res_parameter_v)
 # res_parameter_v[1,0], delta_f = do_resilient_f(prev_err_der_res, now_err_der_res, n_p, n_n,delta_f, res_parameter_v)
  
print(E_present_res)
print(res_parameter_v)
  
  

[[0.92803065]
 [0.96966084]]
0.36074403333203164
[[0.92803065]
 [0.96966084]]


gradient clipping test

In [0]:
f_0 = 1
np.random.seed(1)
#gc_parameter_v = np.random.randint(10,size=(2,1))
#gc_parameter_v = 2*np.random.rand(2,1)
print(gc_parameter_v)
gc_input = makeinput(random_x_test,0,f_0)
E_present_gc = 10
E_previous_gc = 15
#threshold_n = 2
threshold_n = 1
learningrate_gc = 0.00001


#main loop
#while(np.abs(E_present_gc-E_previous_gc)>e):
#for epoch in range(0,5):  
  gc_f_t_arr = get_f_t_arr(f_0, random_x_test, gc_parameter_v, gc_input)
  gc_y_hat = doforwardpropagation(f_0,random_x_test, gc_parameter_v, gc_input)
  
  E_previous_gc = E_present_gc
  E_present_gc = geterr(gc_y_hat,random_y_test)
  
  #err_der_gc = do_err_derivative(random_x_test, gc_f_t_arr, gc_parameter_v, gc_y_hat,random_y_test)
  #gc_parameter_v[0,0] = do_gradient_clip_x(err_der_gc, threshold_n, gc_parameter_v, learningrate_gc)
  #gc_parameter_v[1,0] = do_gradient_clip_f(err_der_gc, threshold_n, gc_parameter_v, learningrate_gc)
  
  print(E_present_gc)
  print(gc_parameter_v)
  
  

In [0]:
x = np.random.binomial(n=1,p=0.5,size=(2,8))
y = x.sum(axis=1).reshape(-1,1)
x,y

(array([[0, 1, 0, 1, 1, 0, 1, 0],
        [1, 0, 1, 0, 1, 0, 0, 0]]), array([[4],
        [3]]))

In [0]:
f_0 = 1
a = makeinput(x ,0 , f_0)
f_0_arr = initialize_f_0(f_0,x)
b = get_f_t_arr(f_0,x,parameter_v,a)
print(b)
c = doforwardpropagation(f_0,x,parameter_v,a)
print(c)

f_1 = get_f_t(a,parameter_v)
print(f_1)
e = updateinput(x,1,f_1)
print(e)
f_2 = get_f_t(e,parameter_v)
print(f_2)

[[1.         0.92795586 1.62089353 1.50411764 2.15554624 2.76004321
  2.56119826 3.13647039 2.91050606]
 [1.         1.68774732 1.56615501 2.21311417 2.05367225 2.66550865
  2.47347436 2.29527502 2.12991389]]
[[2.91050606]
 [2.12991389]]
[[0.92795586]
 [1.68774732]]
[[1.         0.92795586]
 [0.         1.68774732]]
[[1.62089353]
 [1.56615501]]


In [0]:
f_0 = random.random()
f_0_column = np.full((len(input_arr),1),f_0)
f_0_column
new_arr = input_arr[:,0].reshape(-1,1)
ininput_arr = np.hstack([new_arr , f_0_column])
ininput_arr

In [0]:
delta = np.array([1, 1])
a = np.array([1,2])
b = np.array([-3,5])
res_parameter_v = 2*np.random.rand(2,1)
print(res_parameter_v)

delta_x,delta_f = update_delta(a,b,1,1,1.2,0.5)
print(delta_x,delta_f)
res_parameter_v[0,0],delta_x= do_resilient_x(a, b, 1.2, 0.5,delta_x, res_parameter_v)
res_parameter_v[0,0]


[[0.88261845]
 [0.31661974]]
0.5 1.2


1.1326184457919062

In [0]:
'''
def initialize_f_0(f_0,input_arr):
  f_0_column = np.full((len(input_arr),1),f_0)
  return f_0_column

def makeinput(input_arr,index,f_0):
  f_t=initialize_f_0(f_0,input_arr)
  one_column = np.ones((len(input_arr),1)) 
  x_mat_vector = input_arr[:,index]
  x_mat = x_mat_vector.reshape(-1,1)
  concat_mat_1 = np.hstack([one_column,x_mat])
  concat_mat_2 = np.hstack([concat_mat_1,f_t])
  return concat_mat_2

def updateinput(input_arr,index,f_t_arr):
  one_column = np.ones((len(input_arr),1)) 
  x_mat_vector = input_arr[:,index]
  x_mat = x_mat_vector.reshape(-1,1)
  concat_mat_1 = np.hstack([one_column,x_mat])
  concat_mat_2 = np.hstack([concat_mat_1,f_t_arr])
  return concat_mat_2

def get_f_t(input_arr , parameter_v):
  f_t = np.matmul(input_arr, parameter_v)
  return f_t

def get_f_t_arr(f_0,initial_arr,parameter_v,input_arr):
  f_0_arr=np.array([f_0])
  for t in range(1,len(input_arr[0])):
    f_t = get_f_t(initial_arr , parameter_v)
    input_t_arr = updateinput(input_arr, t, f_t)
    f_t_arr = np.append(f_0_arr,f_t)
  f_t_final = get_f_t(input_t_arr,parameter_v)
  f_t_arr = np.append(f_t_arr,f_t_final)
  return  f_t_arr

def doforwardpropagation(initial_arr , parameter_v, input_arr):
  for t in range(1,len(input_arr[0])):
    f_t = get_f_t(initial_arr , parameter_v)
    input_t_arr = updateinput(input_arr, t, f_t)
  f_t_final = get_f_t(input_t_arr , parameter_v)
  y_hat = 1*f_t_final
  return y_hat  

def do_err_derivative(input_arr,f_t_arr, parameter_v, y_hat, y , alpha ):
  sum_t = np.zeros(2) 
  for n_1_k in range(0,2):
    for t in range(0,len(input_arr[0])):
      sum_i = 0
      for i in range(0,len(input_arr)):
        if n_1_k == 0:
          sum_i +=  (y_hat[i,0]-y[i,0])*input_arr[i,t]
        else:
          sum_i +=  (y_hat[i,0]-y[i,0])*f_t_arr[i,t-1]
      sum_t[n_1_k] += sum_i*parameter_v[2,0]**(len(input_arr[0])-t)
  err_der_arr = sum_t
  return err_der_arr    

def do_backpropagation(parameter_v, alpha, err_derivative):
  for n_1_k in range(1,len(parameter_v)):
    parameter_v[n_1_k,0] =   parameter_v[n_1_k,0] - alpha*err_derivative[n_1_k,0]
  return parameter_v  
  
def update_delta(prev_err_der,err_der, delta, n_p , n_n):
  if np.sign(prev_err_der) == np.sign(err_der):
    new_delta[0,0] = n_p*delta[0,0]
  else:
    new_delta[1,0] = n_n*delta[1,0]
  return new_delta
  
def do_resilient_x(prev_err_der , err_der , new_delta , parameter_v):
  for n_1_k in range(1,len(parameter_v)):
    parameter_v[n_1_k,0] = parameter_v[n_1_k,0] - new_delta[n_1_k-1,0]*np.sign(err_derivative[n_1_l,0])
  return parameter_v
  
def do_gradient_clip(err_der , threshold_n, parameter_v ,alpha):
  for n_1_k in range(1,len(parameter_v)):
    if np.abs(err_der[n_1_k,0]) > threshold_n:
      new_err_der[n_1_k,0] = err_der[n_1_k,0]*threshold_n/np.abs(err_der[n_1_k,0])
    else:
      new_err_der[n_1_k,0] = err_der[n_1_k,0]
  new_parameter_v = do_backpropagation(parameter_v , alpha , new_err_der)  
  return new_parameter_v

def geterr(y_hat, y):
  value1 = 0
  for i in range(0,len(y)):
    value1 += (y_hat[i,0]-y[i,0])**2
  err = 0.5*value1    
  return err
 '''   